In [1]:
import json
import pandas as pd

# Load the JSON file, assuming it is properly formatted
data = pd.read_json('spordiehitised.json')

# Function to parse the JSON-like string and extract information
def extract_info(json_str):
    try:
        # Convert to string if not already a string (handling non-string cases)
        if not isinstance(json_str, str):
            json_str = str(json_str)
        
        # Replace single quotes with double quotes for JSON compatibility
        json_str = json_str.replace("'", '"').replace('\\', '\\\\')
        
        # Parse the JSON string
        data_dict = json.loads(json_str)
        # Extract the first key and associated data
        first_key = list(data_dict.keys())[0]
        info = data_dict[first_key]
        
        # Extract required fields
        objektid = info.get('objektid', '')
        objektnimi = info.get('objektnimi', '')
        tehnseisukord = info.get('tehnseisukord', '')
        
        # Extract 'spordialad', handling nested dictionary keys
        spordialad_keys = info.get('spordialad', {}).keys()
        spordialad = ', '.join([info['spordialad'][key].get('spordiala', '') for key in spordialad_keys])
        
        return pd.Series([objektid, objektnimi, tehnseisukord, spordialad])
    except Exception as e:
        # Log error details for review
        with open('error_log.txt', 'a') as f:
            f.write(f"Error processing JSON: {e} - Data: {json_str[:100]}\n")
        return pd.Series([None, None, None, None])

# Define new columns and initialize them
new_columns = ['objektid', 'objektnimi', 'tehnseisukord', 'spordialad']
data[new_columns] = pd.DataFrame(columns=new_columns)  # Initialize the new columns

# Apply the function to the 'paigad' column of the whole dataset
data[new_columns] = data['paigad'].apply(extract_info)

columns_to_keep = [
    'rajatisid', 'rajatisnimi', 'kompleks', 'liik', 'omandiliik', 'ehstaatus',
    'om_oigvorm_id', 'omaniknimi', 'omanikregkood', 'va_oigvorm_id',
    'valdajanimi', 'valdajaregkood', 'kaart_laius', 'kaart_pikkus', 'maakond',
    'maakond_kood', 'kov', 'asustusyksus', 'asustusyksus_kood', 'riietusruumidemahutavus',
    'objektid', 'objektnimi', 'tehnseisukord', 'spordialad'
]

data = data[columns_to_keep]

# save the updated DataFrame to a new CSV file
data.to_csv('spordiehitised.csv', index=False)


  objektid                      objektnimi tehnseisukord  \
0    10849                         Jõusaal      Väga hea   
1    11527                   Aeroobikasaal      Väga hea   
2    10320  24-7fitness Kuressaare jõusaal      Väga hea   
3    10315                         Jõusaal      Väga hea   
4    12348                         Jõusaal      Väga hea   

                                          spordialad  
0      jõutõstmine, Kulturism ja fitness, Tõstesport  
1  aeroobika ja rühmatreeningud, harrastusvõimlemine  
2      jõutõstmine, Kulturism ja fitness, Tõstesport  
3      jõutõstmine, Kulturism ja fitness, Tõstesport  
4      jõutõstmine, Kulturism ja fitness, Tõstesport  


In [2]:
processed_data = pd.read_csv('spordiehitised.csv')

In [3]:
# Convert all string columns to lowercase
processed_data = processed_data.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

processed_data = processed_data.fillna(0)


In [4]:
for column in processed_data.columns:
    if processed_data[column].dtype == 'object':
        processed_data[column] = processed_data[column].astype(str)

columns_to_convert = [
    'omanikregkood', 'valdajaregkood', 'maakond_kood', 'asustusyksus_kood', 'riietusruumidemahutavus',
    'objektid'
]

for column in columns_to_convert:
    processed_data[column] = pd.to_numeric(processed_data[column], errors='coerce').fillna(0).astype(int)

In [12]:
data_org = pd.read_json('spordiorganisatsioonid.json')
columns_to_keep_org = [
    'org_id', 'nimi', 'registrikood', 'maakond', 'maakond_kood', 'kov', 'kov_kood', 'kaart_laius', 'kaart_pikkus'
]
data_org = data_org[columns_to_keep_org]

# save the updated DataFrame to a new CSV file
data_org.to_csv('spordiorganisatsioonid.csv')

processed_data_org = pd.read_csv('spordiorganisatsioonid.csv')

# Convert all string columns to lowercase
processed_data_org = processed_data_org.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

processed_data_org = processed_data_org.fillna(0)

for column in processed_data_org.columns:
    if processed_data_org[column].dtype == 'object':
        processed_data_org[column] = processed_data_org[column].astype(str)

columns_to_convert = [
    'registrikood', 'maakond_kood', 'kov_kood'
]

for column in columns_to_convert:
    processed_data_org[column] = pd.to_numeric(processed_data_org[column], errors='coerce').fillna(0).astype(int)